Web crawl of all names of garden plant

In [55]:
from bs4 import BeautifulSoup
import requests
import pickle
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import re

The page link is analyzed using beautiful soup. All names are included in a div section with class equal to columns.

In [5]:
page_link = 'https://en.wikipedia.org/wiki/List_of_garden_plants'
response = requests.get(page_link)
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
list_items = soup.find_all('div', {'class': 'columns'})

In [11]:
res = []
for item in list_items:
    x = item.find_all('li')
    res.extend([t.text for t in x])

In [38]:
res[:5]

['Abelia',
 'Abeliophyllum (white forsythia)',
 'Abelmoschus (okra)',
 'Abies (fir)',
 'Abroma']

Some of the plants names include the common used daily names, which are labelled in brackets. Anything included in brackets will be deleted/removed.

In [31]:
res_removed = [i[:(i.find('(') - 1)] if i.find('(') != -1 else i for i in res]
res_removed[:5]

['Abelia', 'Abeliophyllum', 'Abelmoschus', 'Abies', 'Abroma']

The size of the data and 

In [42]:
data_lower = list(map(lambda x: x.lower(), res_removed))
chunks = ''.join(list(map(str, data_lower)))
chars = list(set(chunks))
data = copy.deepcopy(chunks)
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 20681 total characters and 32 unique characters in your data.


In [54]:
data.find('syn.')

6237

In [46]:
data_lower[-5:]

['zephyranthes', 'zigadenus', 'zinnia', 'zizania', 'zygopetalum']

In [ ]:
import keras
import tensorflow as tf
